In [1]:
import tensorflow as tf
from keras import layers
from tensorflow.keras import applications 
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras import models

from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix
from tqdm import tqdm

In [2]:
import numpy
dr_images = numpy.load('../input/dr-train-aptos-test/x_train.npy')
dr_labels = numpy.load('../input/dr-train-aptos-test/y_train.npy').argmax(axis=1)
aptos_images = numpy.load('../input/dr-train-aptos-test/x_test.npy')
aptos_labels = numpy.load('../input/dr-train-aptos-test/y_test.npy').argmax(axis=1)

In [11]:
class Metrics(Callback):
    def __init__(self, xval, yval):
        super().__init__()
        self.xval = xval
        self.yval = yval
        
    def on_train_begin(self, logs={}):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        X_val = self.xval
        y_val = self.yval
        
        y_pred = self.model.predict(X_val)
        y_pred = numpy.clip(y_pred,0,4)
        y_pred = y_pred.astype(int)

        _val_kappa = cohen_kappa_score(
            y_val,
            y_pred, 
            weights='quadratic'
        )

        self.val_kappas.append(_val_kappa)

        print(f"val_kappa: {_val_kappa:.4f}")
        
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save('model.h5')

        return
    
kappa_metrics = Metrics(aptos_images, aptos_labels)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=60,
    shear_range=0.3,
    vertical_flip=True,
    horizontal_flip=True)
train_datagen.fit(images)

test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=60,
    shear_range=0.3,
    vertical_flip=True,
    horizontal_flip=True)
test_datagen.fit(xyz)

In [4]:
xception = applications.Xception(include_top=False, weights='imagenet')

model = Sequential()
model.add(xception)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(256))
model.add(layers.Dense(256))
model.add(layers.Dense(1))

83689472/83683744 [==============================] - 1s 0us/step


In [5]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

xception.trainable = False
model.compile(loss='mse', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

for train_index, test_index in skf.split(dr_images, dr_labels):
    x_train, x_test = dr_images[train_index], dr_images[test_index]
    y_train, y_test = dr_labels[train_index], dr_labels[test_index]
    
    history = model.fit(x_train, y_train, batch_size=32, epochs=16, callbacks=[kappa_metrics],
                    validation_data = (x_test, y_test))

Epoch 1/16
84/84 [==============================] - 11s 71ms/step - loss: 229.6167 - accuracy: 0.2673 - val_loss: 6.3532 - val_accuracy: 0.2491
val_kappa: 0.0151
Validation Kappa has improved. Saving model.
Epoch 2/16
84/84 [==============================] - 5s 55ms/step - loss: 30.2178 - accuracy: 0.2609 - val_loss: 5.2602 - val_accuracy: 0.2446
val_kappa: 0.0533
Validation Kappa has improved. Saving model.
Epoch 3/16
84/84 [==============================] - 5s 56ms/step - loss: 13.7431 - accuracy: 0.2669 - val_loss: 2.4977 - val_accuracy: 0.2386
val_kappa: 0.1320
Validation Kappa has improved. Saving model.
Epoch 4/16
84/84 [==============================] - 5s 55ms/step - loss: 8.4388 - accuracy: 0.2369 - val_loss: 3.4026 - val_accuracy: 0.2349
val_kappa: 0.1238
Epoch 5/16
84/84 [==============================] - 5s 55ms/step - loss: 5.1442 - accuracy: 0.2480 - val_loss: 3.4480 - val_accuracy: 0.2666
val_kappa: 0.0538
Epoch 6/16
84/84 [==============================] - 5s 55ms/step 

In [6]:
from keras.models import load_model
model = load_model('./model.h5')

In [6]:
xception.trainable = True
model.compile(loss='mse', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

for train_index, test_index in skf.split(dr_images, dr_labels):
    x_train, x_test = dr_images[train_index], dr_images[test_index]
    y_train, y_test = dr_labels[train_index], dr_labels[test_index]
    
    history = model.fit(x_train, y_train, batch_size=32, epochs=16, callbacks=[kappa_metrics],
                    validation_data = (x_test, y_test))

Epoch 1/16
84/84 [==============================] - 18s 168ms/step - loss: 2.2318 - accuracy: 0.2435 - val_loss: 4.2800 - val_accuracy: 0.2353
val_kappa: 0.3696
Validation Kappa has improved. Saving model.
Epoch 2/16
84/84 [==============================] - 13s 154ms/step - loss: 0.8309 - accuracy: 0.2804 - val_loss: 1.8740 - val_accuracy: 0.2353
val_kappa: 0.6068
Validation Kappa has improved. Saving model.
Epoch 3/16
84/84 [==============================] - 13s 153ms/step - loss: 0.4692 - accuracy: 0.3462 - val_loss: 1.4961 - val_accuracy: 0.2375
val_kappa: 0.7329
Validation Kappa has improved. Saving model.
Epoch 4/16
84/84 [==============================] - 13s 154ms/step - loss: 0.3285 - accuracy: 0.3881 - val_loss: 1.5169 - val_accuracy: 0.2423
val_kappa: 0.7737
Validation Kappa has improved. Saving model.
Epoch 5/16
84/84 [==============================] - 13s 154ms/step - loss: 0.2560 - accuracy: 0.4014 - val_loss: 1.2339 - val_accuracy: 0.2618
val_kappa: 0.7252
Epoch 6/16
84/8

In [ ]:
from keras.models import load_model
best_model = load_model('./model.h5')

In [10]:
from sklearn.model_selection import train_test_split
apx_train, apx_test, apy_train, apy_test = train_test_split(aptos_images, aptos_labels, test_size=0.2,
                                                           random_state=42, shuffle=True, stratify=aptos_labels)

In [12]:
xception.trainable = False
model.compile(loss='mse', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

for train_index, test_index in skf.split(apx_train, apy_train):
    x_train, x_test = apx_train[train_index], apx_train[test_index]
    y_train, y_test = apy_train[train_index], apy_train[test_index]
    
    history = model.fit(x_train, y_train, batch_size=32, epochs=16, callbacks=[kappa_metrics],
                    validation_data = (x_test, y_test))

Epoch 1/16
46/46 [==============================] - 6s 79ms/step - loss: 0.8930 - accuracy: 0.2602 - val_loss: 0.7972 - val_accuracy: 0.3072
val_kappa: 0.5878
Validation Kappa has improved. Saving model.
Epoch 2/16
46/46 [==============================] - 3s 55ms/step - loss: 0.7474 - accuracy: 0.3078 - val_loss: 0.7932 - val_accuracy: 0.3215
val_kappa: 0.7023
Validation Kappa has improved. Saving model.
Epoch 3/16
46/46 [==============================] - 3s 55ms/step - loss: 0.8312 - accuracy: 0.3335 - val_loss: 0.6884 - val_accuracy: 0.3952
val_kappa: 0.6248
Epoch 4/16
46/46 [==============================] - 2s 55ms/step - loss: 0.6943 - accuracy: 0.3797 - val_loss: 0.6512 - val_accuracy: 0.4041
val_kappa: 0.6719
Epoch 5/16
46/46 [==============================] - 3s 57ms/step - loss: 0.7081 - accuracy: 0.3852 - val_loss: 0.6197 - val_accuracy: 0.4246
val_kappa: 0.6851
Epoch 6/16
46/46 [==============================] - 3s 55ms/step - loss: 0.6356 - accuracy: 0.4289 - val_loss: 0.60

In [13]:
xception.trainable = True
model.compile(loss='mse', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

for train_index, test_index in skf.split(apx_train, apy_train):
    x_train, x_test = apx_train[train_index], apx_train[test_index]
    y_train, y_test = apy_train[train_index], apy_train[test_index]
    
    history = model.fit(x_train, y_train, batch_size=32, epochs=16, callbacks=[kappa_metrics],
                    validation_data = (x_test, y_test))

Epoch 1/16
46/46 [==============================] - 12s 177ms/step - loss: 0.6530 - accuracy: 0.4997 - val_loss: 0.4498 - val_accuracy: 0.5672
val_kappa: 0.7502
Validation Kappa has improved. Saving model.
Epoch 2/16
46/46 [==============================] - 7s 155ms/step - loss: 0.2292 - accuracy: 0.5889 - val_loss: 0.4521 - val_accuracy: 0.5625
val_kappa: 0.7533
Validation Kappa has improved. Saving model.
Epoch 3/16
46/46 [==============================] - 7s 155ms/step - loss: 0.1230 - accuracy: 0.5906 - val_loss: 0.4197 - val_accuracy: 0.5584
val_kappa: 0.8733
Validation Kappa has improved. Saving model.
Epoch 4/16
46/46 [==============================] - 7s 155ms/step - loss: 0.0919 - accuracy: 0.5865 - val_loss: 0.3947 - val_accuracy: 0.5666
val_kappa: 0.8102
Epoch 5/16
46/46 [==============================] - 7s 154ms/step - loss: 0.0673 - accuracy: 0.5966 - val_loss: 0.4015 - val_accuracy: 0.5706
val_kappa: 0.8759
Validation Kappa has improved. Saving model.
Epoch 6/16
46/46 [=

In [8]:
def predict(X, coef):
    X_p = numpy.copy(X)
    for i, pred in enumerate(X_p):
        if pred < coef[0]:
            X_p[i] = 0
        elif pred >= coef[0] and pred < coef[1]:
            X_p[i] = 1
        elif pred >= coef[1] and pred < coef[2]:
            X_p[i] = 2
        elif pred >= coef[2] and pred < coef[3]:
            X_p[i] = 3
        else:
            X_p[i] = 4
    return X_p

In [14]:
from keras.models import load_model
best_model = load_model('./model.h5')

from sklearn.metrics import confusion_matrix
pred = predict(best_model.predict(apx_test), [0.5, 1.5, 2.5, 3.5])
confusion_matrix(apy_test, pred)

array([[344,  13,   4,   0,   0],
       [ 10,  28,  33,   3,   0],
       [  2,  10, 151,  34,   3],
       [  0,   1,   8,  25,   5],
       [  0,   2,  20,  20,  17]])